# Call the scattering solutions & compare profiles

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import scattering_for_kees as scattering

Set parameters

In [ ]:
eps_e = 0.5
tau   = 3
T_fg  = 200.       # dust temperature [K]
T_bg  = 1e-2# 2.7        # background temperature [K]

In [ ]:
lam = 0.1
nu  = scattering.c_light / lam
Bnu = scattering.bplanck(nu, T_fg)

Calculate RADMC-3D result

In [ ]:
print(f'Setup:\n------\n  tau = {tau:.2g}\n  eps = {eps_e:.2g}')
res = scattering.radmc3d_scattering_solution(
    tau, eps_e, T_fg, T_bg, keep_folder=True)

tau_RMC = np.cumsum(res['rho'] * np.diff(res['zi']))

## Plotting

Calculate analytical results

In [ ]:
tauz = np.linspace(0, tau, 1000)

JoB = scattering.J_over_B(tauz, eps_e, tau)
SoB = scattering.S_over_B(tauz, eps_e, tau)
IoB = scattering.I_over_B(eps_e, tau)
I_o = scattering.I_over_B_out(tau, eps_e)

How close are we to the right gradient at the boundary

In [ ]:
print(f'analytic gradient / sqrt(3) = {np.gradient(JoB, tauz)[0]/np.mean(JoB[:2]) / np.sqrt(3)}')
print(f'RADMC-3D gradient / sqrt(3) = {np.gradient(res["Jnu"], tau_RMC)[0]/np.mean(res["Jnu"][:2]) / np.sqrt(3)}')

In [ ]:
f, ax = plt.subplots(dpi=150)

ax.semilogy(tau_RMC, res['Jnu']/Bnu, 'C1--', label=r'$J_\nu / B_\nu$ (RADMC-3D)')
ax.semilogy(tauz, JoB, 'C1', label=r'$J_\nu / B_\nu$ (analytic)')

ax.plot(tauz, SoB, 'C2--', label=r'$S_\nu / B_\nu$ (analytic)')

ax.axhline(res['I_out'] / Bnu, c='k', ls='--', lw=3, label=r'$I_{\nu,out} / B_\nu$ (RADMC-3D)')
ax.axhline(I_o, c='k', ls='-', label=r'$I_{\nu,out} / B_\nu$ (analytic)')
ax.axhline(IoB, c='0.7', ls=':', lw=3, label=r'$I_{\nu,out} / B_\nu$ (analytic, E-B)')
ax.legend(handlelength=3);
ax.set_ylim(1e-1, 1e0)